In [ ]:
import numpy as np
import h5py
import setGPU

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation, Layer, Reshape
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras import backend as K
import math
import pickle

from datetime import datetime
import os
import tensorflow_model_optimization as tfmot
from qkeras import QDense, QActivation
from qkeras import *

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from functions import preprocess_anomaly_data, custom_loss_negative, custom_loss_training,\
roc_objective,load_model, save_model
from custom_layers import Sampling

from autoencoder_classes import AE

tsk = tfmot.sparsity.keras

In [ ]:
# Data = (N,19,3,1).flatten()
with open('/eos/user/e/epuljak/forDelphes/Delphes_QCD_BSM_data.pkl', 'rb') as f:
    X_train_flatten, X_train_scaled, X_test_flatten, X_test_scaled, bsm_data, bsm_target, pt_scaler = pickle.load(f)

In [ ]:
quant_size = 16
integer = 6
symmetric = 1
alpha=1
pruning='pruned'

### Define model
Prune and quantize only encoder.

In [ ]:
latent_dim = 3
input_shape = 57

In [ ]:
#encoder
inputArray = Input(shape=(input_shape))
x = Activation('linear')(inputArray) if quant_size==0\
   else QActivation(f'quantized_bits(16,10,1, alpha=1.0)')(inputArray)
x = BatchNormalization()(x)
x = Dense(32, kernel_initializer=tf.keras.initializers.HeUniform())(x) if quant_size==0\
    else QDense(32, kernel_initializer=tf.keras.initializers.HeUniform(),\
               kernel_quantizer='quantized_bits(' + str(quant_size) + ',' + str(integer) + ',1, alpha=1.0)',
               bias_quantizer='quantized_bits(' + str(quant_size) + ',' + str(integer) + ',1, alpha=1.0)')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x) if quant_size==0\
    else QActivation('quantized_relu(bits=' + str(quant_size) + ')')(x)
x = Dense(16, kernel_initializer=tf.keras.initializers.HeUniform())(x) if quant_size==0\
    else QDense(16, kernel_initializer=tf.keras.initializers.HeUniform(),\
               kernel_quantizer='quantized_bits(' + str(quant_size) + ',' + str(integer) + ',1, alpha=1.0)',
               bias_quantizer='quantized_bits(' + str(quant_size) + ',' + str(integer) + ',1, alpha=1.0)')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x) if quant_size==0\
    else QActivation('quantized_relu(bits=' + str(quant_size) + ')')(x)
encoder = Dense(latent_dim, kernel_initializer=tf.keras.initializers.HeUniform())(x) if quant_size==0\
    else QDense(latent_dim, kernel_initializer=tf.keras.initializers.HeUniform(),\
               kernel_quantizer='quantized_bits(' + str(quant_size) + ',' + str(integer) + ',1, alpha=1.0)',\
               bias_quantizer='quantized_bits(' + str(quant_size) + ',' + str(integer) + ',1, alpha=1.0)')(x)
#x = BatchNormalization()(x)
#encoder = tsk.prune_low_magnitude(LeakyReLU(alpha=0.3),pruning_schedule=pruning_schedule)(x) if quant_size==0\
    #else tsk.prune_low_magnitude(QActivation('quantized_relu(bits=' + str(quant_size) + ')'),pruning_schedule=pruning_schedule)(x)

#decoder
x = Dense(16, kernel_initializer=tf.keras.initializers.HeUniform())(encoder) if quant_size==0\
    else QDense(16, kernel_initializer=tf.keras.initializers.HeUniform(),\
               kernel_quantizer='quantized_bits(' + str(quant_size) + ',' + str(integer) + ',1, alpha=1.0)',
               bias_quantizer='quantized_bits(' + str(quant_size) + ',' + str(integer) + ',1, alpha=1.0)')(encoder)
x = BatchNormalization()(x)
x = Activation('relu')(x) if quant_size==0\
    else QActivation('quantized_relu(bits=' + str(quant_size) + ')')(x)
x = Dense(32, kernel_initializer=tf.keras.initializers.HeUniform())(x) if quant_size==0\
    else QDense(32, kernel_initializer=tf.keras.initializers.HeUniform(),\
               kernel_quantizer='quantized_bits(' + str(quant_size) + ',' + str(integer) + ',1, alpha=1.0)',\
               bias_quantizer='quantized_bits(' + str(quant_size) + ',' + str(integer) + ',1, alpha=1.0)')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x) if quant_size==0\
    else QActivation('quantized_relu(bits=' + str(quant_size) + ')')(x)
decoder = Dense(input_shape, kernel_initializer=tf.keras.initializers.HeUniform(),  name='output_dense')(x) if quant_size==0\
        else QDense(input_shape, kernel_initializer=tf.keras.initializers.HeUniform(),\
               kernel_quantizer='quantized_bits(' + str(16) + ',' + str(10) + ',1, alpha=1.0)',\
               bias_quantizer='quantized_bits(' + str(16) + ',' + str(10) + ',1, alpha=1.0)')(x)

#create autoencoder
autoencoder = Model(inputs = inputArray, outputs=decoder)
autoencoder.summary()

In [ ]:
ae = AE(autoenc=autoencoder)
ae.compile(optimizer=keras.optimizers.Adam())

In [ ]:
# transfer weights
model_dir = 'AE_models/final_models/withCorrectPrefiltering/'
name_encoder ='AE_pruned'
baseline_AE = load_model(model_dir+name_encoder, custom_objects={'QDense': QDense, 'QActivation': QActivation})

# set weights for encoder
for i, l in enumerate(ae.autoencoder.layers):
    if i < 2: continue
    ae.autoencoder.layers[i].set_weights(baseline_AE.layers[i-1].get_weights())

## Add MSE layer

In [ ]:
from custom_layers import CustomMSE

In [ ]:
true = ae.autoencoder.input
predicted = ae.autoencoder.layers[-1].output
reshaped_predicted = Reshape((19,3,1), name='reshaped_predicted')(predicted)

scaled_input = BatchNormalization(trainable=False, name='scaled_input')(true)
reshaped_scaled_input = Reshape((19,3,1), name='reshaped_scaled_input')(scaled_input)
# calculate MSE between them
custom_output = CustomMSE()([reshaped_scaled_input, reshaped_predicted])
# create new model
model = Model(inputs=ae.autoencoder.input, outputs=custom_output)
""" Keras BatchNorm layer returns
    gamma * (batch - self.moving_mean) / sqrt(self.moving_var + epsilon) + beta
    epsilon=0.001
    momentum=0.99
    moving_mean = moving_mean * momentum + mean(batch) * (1 - momentum)
    moving_variance = moving_var * momentum + var(batch) * (1 - momentum)
    pt_scaler
    pt_scaler.mean_
    pt_scaler.var_
"""
# with open('output/data_-1.pickle', 'rb') as f:
#     x_train, y_train, _, _, _, pt_scaler = pickle.load(f)
mean_ = np.zeros((57,))
var_ = np.ones((57,))
for i in range(19):
    mean_[3*i] = pt_scaler.mean_[i]
    var_[3*i] = pt_scaler.var_[i]
# order of weights is (gamma,beta,mean,std)
model.get_layer('scaled_input').set_weights((np.ones((57,)),np.zeros((57,)),mean_,var_))

In [ ]:
model.summary()

In [ ]:
save_model('AE_models/final_models/withCorrectPrefiltering/PTQ_AE_qkeras14', model)

In [ ]:
model.compile()

## Compare QKeras model and BP model

In [ ]:
with open('/eos/user/e/epuljak/forDelphes/Delphes_QCD_BSM_data_half2.pkl', 'rb') as f:
    X_train_flatten, X_train_scaled, X_test_flatten, X_test_scaled, bsm_data, bsm_target = pickle.load(f)

In [ ]:
# create custom model for BP model

true_BP = baseline_AE.input
predicted_BP = baseline_AE.layers[-1].output
reshaped_predicted_BP = Reshape((19,3,1), name='reshaped_predicted_bp')(predicted_BP)

scaled_input_BP = BatchNormalization(trainable=False, name='scaled_input_bp')(true_BP)
reshaped_scaled_input_BP = Reshape((19,3,1), name='reshaped_scaled_input_bp')(scaled_input_BP)
# calculate MSE between them
custom_output_BP = CustomMSE()([reshaped_scaled_input_BP, reshaped_predicted_BP])
# create new model
model_BP = Model(inputs=baseline_AE.input, outputs=custom_output_BP)
""" Keras BatchNorm layer returns
    gamma * (batch - self.moving_mean) / sqrt(self.moving_var + epsilon) + beta
    epsilon=0.001
    momentum=0.99
    moving_mean = moving_mean * momentum + mean(batch) * (1 - momentum)
    moving_variance = moving_var * momentum + var(batch) * (1 - momentum)
    pt_scaler
    pt_scaler.mean_
    pt_scaler.var_
"""
# with open('output/data_-1.pickle', 'rb') as f:
#     x_train, y_train, _, _, _, pt_scaler = pickle.load(f)
mean_BP = np.zeros((57,))
var_BP = np.ones((57,))
for i in range(19):
    mean_BP[3*i] = pt_scaler.mean_[i]
    var_BP[3*i] = pt_scaler.var_[i]
# order of weights is (gamma,beta,mean,std)
model_BP.get_layer('scaled_input_bp').set_weights((np.ones((57,)),np.zeros((57,)),mean_BP,var_BP))
model_BP.summary()
model_BP.compile()

In [ ]:
y_qkeras = ae.autoencoder.predict(X_test_flatten)
y_BP = baseline_AE.predict(X_test_flatten)

In [ ]:
from functions import make_mse_loss_numpy
mse_loss_total = []
mse_loss_total.append(make_mse_loss_numpy(X_test_flatten,y_qkeras))
mse_loss_total.append(make_mse_loss_numpy(X_test_flatten,y_BP))

In [ ]:
bsm_labels = ['Leptoquark','A to 4 leptons', 'hChToTauNu', 'hToTauTau']
labels = ['QCD QKeras', 'QCD BP Keras',\
          r'QKeras LQ $\rightarrow$ b$\tau$', r'BP Keras LQ $\rightarrow$ b$\tau$',\
          r'QKeras A $\rightarrow$ 4L', r'BP Keras A $\rightarrow$ 4L',\
          r'QKeras $h_{\pm} \rightarrow \tau\nu$', r'BP Keras $h_{\pm} \rightarrow \tau\nu$',\
          r'QKeras $h_{0} \rightarrow \tau\tau$', r'BP Keras $h_{0} \rightarrow \tau\tau$']
loss = '$MSE$'

colors = ['C1','C2', 'C3', 'C4', 'C5', 'C6']

In [ ]:
for i, label in enumerate(bsm_labels):
    qkeras_pred = ae.autoencoder.predict(bsm_data[i])
    BP_pred = baseline_AE.predict(bsm_data[i])
    
    mse_loss_total.append(make_mse_loss_numpy(bsm_data[i],qkeras_pred))
    mse_loss_total.append(make_mse_loss_numpy(bsm_data[i],BP_pred))
    print("========================================================================")

In [ ]:
minScore = 999999.
maxScore = 0
for i in range(len(labels)):
    thisMin = np.min(mse_loss_total[i])
    thisMax = np.max(mse_loss_total[i])
    minScore = min(thisMin, minScore)
    maxScore = max(maxScore, thisMax)

In [ ]:
# plot loss distributions
bin_size=100
plt.figure(figsize=(10,8))
z = 0
for i, label in enumerate(labels):
    if i%2==0:
        plt.hist(mse_loss_total[i+2].reshape(mse_loss_total[i+2].shape[0]*1), bins=bin_size, label=label, density = True, range=(minScore, 10),
         histtype='step', fill=False, linewidth=1.5, color=colors[z])
    if i%2==1:
        plt.hist(mse_loss_total[i+2].reshape(mse_loss_total[i+2].shape[0]*1), bins=bin_size, label=label, density = True, range=(minScore, 10),
         histtype='step', fill=False, linewidth=1.5, alpha=0.6, color=colors[z])
        z = z+1
#plt.semilogx()
plt.semilogy()
plt.xlabel("Loss")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
plt.title('KL loss')
plt.legend(loc='best')
plt.show()

In [ ]:
# store data for ROCs
from sklearn.metrics import roc_curve, auc
tpr_lq=[];fpr_lq=[];auc_lq=[]
tpr_ato4l=[];fpr_ato4l=[];auc_ato4l=[]
tpr_ch=[];fpr_ch=[];auc_ch=[]
tpr_to=[];fpr_to=[];auc_to=[]


target_qcd_qkeras = np.zeros(mse_loss_total[0].shape[0])
target_qcd_BP = np.zeros(mse_loss_total[1].shape[0])

for i, label in enumerate(labels):
    if i == 0 and i==1: continue
    if i%2==0:
        trueVal = np.concatenate((np.ones(mse_loss_total[i].shape[0]), target_qcd_qkeras))
        predVal_loss = np.concatenate((mse_loss_total[i], mse_loss_total[0]))

        fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

        auc_loss = auc(fpr_loss, tpr_loss)
        if i==2:
            tpr_lq.append(tpr_loss)
            fpr_lq.append(fpr_loss)
            auc_lq.append(auc_loss)
        elif i == 4:
            tpr_ato4l.append(tpr_loss)
            fpr_ato4l.append(fpr_loss)
            auc_ato4l.append(auc_loss)
        elif i==6:
            tpr_ch.append(tpr_loss)
            fpr_ch.append(fpr_loss)
            auc_ch.append(auc_loss)
        elif i == 8:
            tpr_to.append(tpr_loss)
            fpr_to.append(fpr_loss)
            auc_to.append(auc_loss)
    if i%2==1:
        
        trueVal = np.concatenate((np.ones(mse_loss_total[i].shape[0]), target_qcd_BP))
        predVal_loss = np.concatenate((mse_loss_total[i], mse_loss_total[1]))

        fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

        auc_loss = auc(fpr_loss, tpr_loss)
        if i==3:
            tpr_lq.append(tpr_loss)
            fpr_lq.append(fpr_loss)
            auc_lq.append(auc_loss)
        elif i == 5:
            tpr_ato4l.append(tpr_loss)
            fpr_ato4l.append(fpr_loss)
            auc_ato4l.append(auc_loss)
        elif i==7:
            tpr_ch.append(tpr_loss)
            fpr_ch.append(fpr_loss)
            auc_ch.append(auc_loss)
        elif i == 9:
            tpr_to.append(tpr_loss)
            fpr_to.append(fpr_loss)
            auc_to.append(auc_loss)

In [ ]:
# create ROCs
plt.figure(figsize=(12,8))
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_lq[:], fpr_lq[:], auc_lq[:], labels[2:4])):
    if i == 1:
        plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[0], alpha=0.6)
    else: 
        plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[0])

for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_ato4l[:], fpr_ato4l[:], auc_ato4l[:], labels[4:6])):
    if i == 1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[1], alpha = 0.6)
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[1])
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_ch[:], fpr_ch[:], auc_ch[:], labels[6:8])):
    if i==1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[2], alpha=0.6)
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[2])

for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_to[:], fpr_to[:], auc_to[:], labels[8:])):
    if i==1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[3], alpha=0.6)
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[3])
plt.semilogx()
plt.semilogy()
plt.ylabel("True Positive Rate", fontsize=15)
plt.xlabel("False Positive Rate", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
#plt.ylim(0,0.5)
plt.legend(bbox_to_anchor=[1.2, 0.5],loc='best',frameon=True)
plt.tight_layout()
plt.plot(np.linspace(0, 1),np.linspace(0, 1), '--', color='0.75')
plt.axvline(0.00001, color='red', linestyle='dashed', linewidth=1)
#plt.title('QKERAS <16,6>')
plt.show()

In [ ]:
output_result = '/eos/user/e/epuljak/forDelphes/CorrectDataResults/PTQ_AE_result_qkeras14.h5'

In [ ]:
h5f = h5py.File(output_result, 'w')
h5f.create_dataset('QCD_Qkeras', data = mse_loss_total[0])
h5f.create_dataset('QCD_BP', data = mse_loss_total[1])
for i,bsm in enumerate(bsm_labels[:]):
    print(i)
    if i == 0: z = 2
    elif i == 1: z = 4
    elif i == 2: z = 6
    elif i == 3: z = 8
    h5f.create_dataset('%s_Qkeras' %bsm, data = mse_loss_total[z])
    h5f.create_dataset('%s_BP'%bsm, data = mse_loss_total[z+1])
h5f.close()